# Hate_Speech_Detection

In [43]:
import pandas as pd
temp_df = pd.read_csv(r"HateSpeechDatasetBalanced.csv")
temp_df

,Content,Label
0,denial of normal the con be asked to comment o...,1
1,just by being able to tweet this insufferable ...,1
2,that is retarded you too cute to be single tha...,1
3,thought of a real badass mongol style declarat...,1
4,afro american basho,1
...,...,...
726114,i mute this telecasting and played kanye west ...,1
726115,but hell yeah he s not a bachelor but looooooo...,1
726116,great video musician but s not my musician lol...,1
726117,not great pop video yeah he s not a pedophile ...,1


In [44]:
df = temp_df.iloc[:10000]
df.shape

(10000, 2)

# EDA

In [45]:
df['Label'].value_counts()

Label
0    8558
1    1442
Name: count, dtype: int64

In [46]:
df.isnull().sum()

Content    0
Label      0
dtype: int64

In [47]:
df.duplicated().sum()

0

# Data Preprocessing

In [48]:
from string import punctuation
from nltk.corpus import stopwords
stop = stopwords.words("english")
sw_list = list(punctuation) + stop

In [49]:
def text_cleaner(text):
    filtered_text = []
    from nltk.tokenize import word_tokenize
    tokenize = word_tokenize(text)
    for word in tokenize:
        if word.lower() not in sw_list:
            filtered_text.append(word)
    return " ".join(filtered_text)

In [50]:
df['Content'] = df['Content'].apply(text_cleaner)

C:\Users\Acer\AppData\Local\Temp\ipykernel_11216\838255399.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Content'] = df['Content'].apply(text_cleaner)


In [51]:
# 4) differentiating X and y

X = df.iloc[:,0:1]
y = df['Label']

In [52]:
X

,Content
0,denial normal con asked comment tragedies emot...
1,able tweet insufferable bullshit proves trump ...
2,retarded cute single life
3,thought real badass mongol style declaration w...
4,afro american basho
...,...
9995,recent statewide flying campaign blanketed col...
9996,caught stories generated
9997,winning activism
9998,reply joining group hardworking smart activists


In [53]:
y

0       1
1       1
2       1
3       1
4       1
       ..
9995    0
9996    0
9997    0
9998    0
9999    0
Name: Label, Length: 10000, dtype: int64

In [54]:
# Train test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1)

# 5) Feature Extraction

In [60]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
X_train_bow = cv.fit_transform(X_train['Content']).toarray()
X_test_bow = cv.transform(X_test['Content']).toarray()

# 6) Applying Model

In [39]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train_bow,y_train) # fitting training data

GaussianNB()

In [63]:
# predict
y_pred = gnb.predict(X_test_bow)

# metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
print(accuracy_score(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.6355
[[1155  548]
 [ 181  116]]
              precision    recall  f1-score   support

           0       0.86      0.68      0.76      1703
           1       0.17      0.39      0.24       297

    accuracy                           0.64      2000
   macro avg       0.52      0.53      0.50      2000
weighted avg       0.76      0.64      0.68      2000

